# WildObs Tech Demo: Using YoloV11 for Australian Animal Detection

WildObs is a new national platform that will provide and end-to-end solution for wildlife monitoring in Australia. One of its key features is the ability to automatically detect Australian animals in images. For this, the WildObs team is developing the next generation of Australia's animal detection models using existing national and international tools.

In this notebook, we use the powerful image hosting services of the Atlas of Living Australia (ALA) and the new YoloV11 model to demonstrate how to use a base model to detect Australian animals in images. Let's see how good is this model at detecting an iconic Australian animal, the echidna

## Install Required Libraries
For this demo we will need the following libraries:
- `ultralytics` for the YoloV11 model
- `pandas` for data manipulation
- `pathlib` for file manipulation

You can install them using the following command:
```bash
!pip install -q ultralytics pandas pathlib
```


## Download the images
Please see dowload_images.R script to download the images from the ALA. The script will download the images and save them in the `media` folder.

NOTE: The Python Galah library is currently not able to download media from the ALA. An issue has been [raised](https://github.com/AtlasOfLivingAustralia/galah-python/issues/218#issue-2596291312). 

An example of the images downloaded is shown below:

<div style="width: 300px;">
    <img src="00d23640-f155-4aed-8379-b0bf6cf904ac.png" alt="echidna" style="width: 100%;">
</div>


# Load the libraries

In [58]:
import os
from pathlib import Path
from ultralytics import YOLO 
import pandas as pd

## Define our variables

In [59]:
# Define the path to the folder containing the images
image_folder = 'media' 

# Load the YOLOv11 model
model = YOLO('yolo11n.pt')  


## Extract the class names from the base model
When AI models are trained, they are trained on a specific set of classes. In this case, the YoloV11 model was trained on the COCO dataset. We can extract the class names from the model to see what classes it was trained on. We will use this information to visualise the results later

In [60]:
# extract class names from the model
class_names = model.names

# Create a dataframe from the class names
df = pd.DataFrame(list(class_names.items()), columns=['class_number', 'class_name'])

## Analyse the images using the YoloV11 model
We will use the YoloV11 model to analyse the images and detect the echidna. We are using the `predict` function from the `ultralytics` library to do this. The function will return a directory with the images (and its detections overlaid) and a subdirectory with the detections in a text file (called `labels`). This is why we add the parameters `save_txt=True`. 

In [ ]:
model.predict(source=image_folder, save=True, save_txt=True)

## Wrangling the results
We now have the results of the detections in individuals txt files for each image. These txt files do not have column names, and as such we need to add structure to them. We will use the `pandas` library to do this. All YOLO models handle predictions (AI image analysis) the same way: a txt file with one row per detection, where each row has the following format: `class x_center y_center width height`. We will add the column names to the txt files and then concatenate them into a single dataframe.

### Define the directory where the detections are saved
Include also an empty list to append the values from the txt files

In [62]:
labels_dir = 'runs/detect/predict/labels'
all_data = []

In [63]:
# Iterate through each file in the labels directory
for label_file in os.listdir(labels_dir):
    # Check if the file is a .txt file
    if label_file.endswith(".txt"):
        # Read the contents of the label file
        with open(os.path.join(labels_dir, label_file), "r") as file:
            lines = file.readlines()
            for line in lines:
                values = line.split()
                # Add the label file name (without the .txt extension) as the image name
                image_name = label_file.replace('.txt', '')
                values.append(image_name)
                all_data.append(values)

# Create a DataFrame from the combined list, including the image_name column
final_df = pd.DataFrame(all_data, columns=["class_number", "cx", "cy", "width", "height", "image_name"])


### Filter the dataframe and print the results

In [64]:
# Filter the final DataFrame to include only the class number and the image name
filtered_df = final_df[['class_number', 'image_name']]

# Print the filtered DataFrame
print(filtered_df)

### Add the class names to the dataframe
We will use the class names extracted from the YoloV11 model to add the class names to the dataframe. This will allow us to see the class names instead of the class numbers.

In [65]:
# Convert class_number to string in both DataFrames to ensure they match
filtered_df['class_number'] = filtered_df['class_number'].astype(str)
df['class_number'] = df['class_number'].astype(str)

# Perform the merge after ensuring the same data type
final_df = filtered_df.merge(df, on='class_number', how='left')

# Print the resulting DataFrame
print(final_df)

## Explaning the results
The YOLO models was terrible at detecting echidnas. It detected random objects in the images and even detected the echidna as a "bear" (maybe echinas are spikey bears?). This is because the model was trained on a huge dataset with a lot of classes, and echidnas may have not been part of the training set. This is why it is important to train models on specific datasets to get good results; and why the WildObs team is developing a new model specifically for Australian animals.

<div style="width: 300px;">
    <img src="https://github.com/WildObs/ala-computervision-demo/blob/ca21ab6deb1e24ac79df8e0df146274b7599a9e6/yolov11/0a498cce-9acf-49fd-80bc-c31707155105.png" style="width: 300%;">
</div>